## 1.0 Libraries

In [4]:
from IPython.display import display # get ipython for nicer output
import pandas as pd # to build dataframe
from tkinter import * # to have drop.down menues
import numpy as np
from tkinter import ttk

## 1.1 Load Data

In [5]:
df=pd.read_csv('data/Top100.csv')

In [6]:
# merge song title and artist together and write to a list
df['Title - Artist'] = df['Title'].str.cat(others=df['Artist'],sep=' - ')

In [7]:
# Create an alphabetical list of all the available songs
songlist = []
for i in range(len(df['Title - Artist'])):
    songlist.append(df['Title - Artist'][i])
songlist = sorted(songlist)

## 2.1 Regular version

In [168]:
user_input = input("What's you favorite Song:") # user input
user_input = user_input.lower()
if (df['Title'].str.lower() == user_input).any():
    #df_exclude = df.drop(user_input.title(),axis=0)
    print('You might also like:')
    print(df['Title - Artist'].sample(n=1).values[0])
else:
    print('Unfortunately, the song is not in the hot list!')

You might also like:
Message In A Bottle (Taylor's Version) (From The Vault) - Taylor Swift


## 2.2 Tkinter OptionsMenu

In [190]:
root = Tk()
root.title('Song Recommender by Tobias')
root.geometry('400x400')

def output():
    if (df['Title - Artist'] == selected.get()).any():
        out = 'You might also like:\n' + df['Title - Artist'].sample(n=1).values[0]
    else:
        out = 'Unfortunately, the song is not in the hot list!'
    myLabel = Label(root,text=out).pack()

selected = StringVar()

#drop = ttk.Combobox(root, textvariable=selected, values=songlist)
drop = OptionMenu(root, selected, *songlist)
drop.pack()

mybutton = Button(root, text="Select this song!",command=output).pack()

root.mainloop()

In [185]:
selected.get

<bound method StringVar.get of <tkinter.StringVar object at 0x000001BC49E4E0D0>>

## 2.3 Autocomplete Version

In [166]:
user_input = input("Your choice: ")
user_input_low = user_input.lower()
songlist_low = [x.lower() for x in songlist]
#an_iterator = filter(lambda x: x.startswith(user_input_low), songlist_low)
an_iterator = filter(lambda x: user_input_low in x, songlist_low)
filtered_songs = list(an_iterator)

if len(filtered_songs) > 1:
    # Deal with more than one song.
    print('There are more than one song containing "{0}"'.format(user_input))
    print('Select the song from choices: ')
    for index, name in enumerate(filtered_songs):
        print("{0}: {1}".format(index, name))

    index = int(input("Enter choice number: "))
    # You might want to handle IndexError exception here.
    print('\nSelected song: {0}'.format(filtered_songs[index]))
    print('\nYou might also like:\n' + df['Title - Artist'].sample(n=1).values[0])

elif len(filtered_songs) == 1:
    # Only one song found, so print that song.
    print(filtered_songs[0])
    print('\nYou might also like:\n' + df['Title - Artist'].sample(n=1).values[0])
else:
    print('Unfortunately, the song is not in the hot list!')

sand in my boots - morgan wallen

You might also like:
The Very First Night (Taylor's Version) (From The Vault) - Taylor Swift


In [156]:
user_input = input("Your choice: ")
user_input_low = user_input.lower()
songlist_low = [x.lower() for x in songlist]
an_iterator = filter(lambda x: x.startswith(user_input_low), songlist_low)
filtered_songs = list(an_iterator)
print(filtered_songs)

['easy on me - adele']


In [192]:
"""
tkentrycomplete.py

A Tkinter widget that features autocompletion.

Created by Mitja Martini on 2008-11-29.
Updated by Russell Adams, 2011/01/24 to support Python 3 and Combobox.
Updated by Dominic Kexel to use Tkinter and ttk instead of tkinter and tkinter.ttk
   Licensed same as original (not specified?), or public domain, whichever is less restrictive.
"""
import sys
import os
import Tkinter
import ttk

__version__ = "1.1"

# I may have broken the unicode...
Tkinter_umlauts=['odiaeresis', 'adiaeresis', 'udiaeresis', 'Odiaeresis', 'Adiaeresis', 'Udiaeresis', 'ssharp']

class AutocompleteEntry(Tkinter.Entry):
        """
        Subclass of Tkinter.Entry that features autocompletion.

        To enable autocompletion use set_completion_list(list) to define
        a list of possible strings to hit.
        To cycle through hits use down and up arrow keys.
        """
        def set_completion_list(self, completion_list):
                self._completion_list = sorted(completion_list, key=str.lower) # Work with a sorted list
                self._hits = []
                self._hit_index = 0
                self.position = 0
                self.bind('<KeyRelease>', self.handle_keyrelease)

        def autocomplete(self, delta=0):
                """autocomplete the Entry, delta may be 0/1/-1 to cycle through possible hits"""
                if delta: # need to delete selection otherwise we would fix the current position
                        self.delete(self.position, tkinter.END)
                else: # set position to end so selection starts where textentry ended
                        self.position = len(self.get())
                # collect hits
                _hits = []
                for element in self._completion_list:
                        if element.lower().startswith(self.get().lower()):  # Match case-insensitively
                                _hits.append(element)
                # if we have a new hit list, keep this in mind
                if _hits != self._hits:
                        self._hit_index = 0
                        self._hits=_hits
                # only allow cycling if we are in a known hit list
                if _hits == self._hits and self._hits:
                        self._hit_index = (self._hit_index + delta) % len(self._hits)
                # now finally perform the auto completion
                if self._hits:
                        self.delete(0,Tkinter.END)
                        self.insert(0,self._hits[self._hit_index])
                        self.select_range(self.position,Tkinter.END)

        def handle_keyrelease(self, event):
                """event handler for the keyrelease event on this widget"""
                if event.keysym == "BackSpace":
                        self.delete(self.index(Tkinter.INSERT), Tkinter.END)
                        self.position = self.index(Tkinter.END)
                if event.keysym == "Left":
                        if self.position < self.index(Tkinter.END): # delete the selection
                                self.delete(self.position, Tkinter.END)
                        else:
                                self.position = self.position-1 # delete one character
                                self.delete(self.position, Tkinter.END)
                if event.keysym == "Right":
                        self.position = self.index(Tkinter.END) # go to end (no selection)
                if event.keysym == "Down":
                        self.autocomplete(1) # cycle to next hit
                if event.keysym == "Up":
                        self.autocomplete(-1) # cycle to previous hit
                if len(event.keysym) == 1 or event.keysym in Tkinter_umlauts:
                        self.autocomplete()

class AutocompleteCombobox(ttk.Combobox):

        def set_completion_list(self, completion_list):
                """Use our completion list as our drop down selection menu, arrows move through menu."""
                self._completion_list = sorted(completion_list, key=str.lower) # Work with a sorted list
                self._hits = []
                self._hit_index = 0
                self.position = 0
                self.bind('<KeyRelease>', self.handle_keyrelease)
                self['values'] = self._completion_list  # Setup our popup menu

        def autocomplete(self, delta=0):
                """autocomplete the Combobox, delta may be 0/1/-1 to cycle through possible hits"""
                if delta: # need to delete selection otherwise we would fix the current position
                        self.delete(self.position, Tkinter.END)
                else: # set position to end so selection starts where textentry ended
                        self.position = len(self.get())
                # collect hits
                _hits = []
                for element in self._completion_list:
                        if element.lower().startswith(self.get().lower()): # Match case insensitively
                                _hits.append(element)
                # if we have a new hit list, keep this in mind
                if _hits != self._hits:
                        self._hit_index = 0
                        self._hits=_hits
                # only allow cycling if we are in a known hit list
                if _hits == self._hits and self._hits:
                        self._hit_index = (self._hit_index + delta) % len(self._hits)
                # now finally perform the auto completion
                if self._hits:
                        self.delete(0,Tkinter.END)
                        self.insert(0,self._hits[self._hit_index])
                        self.select_range(self.position,Tkinter.END)

        def handle_keyrelease(self, event):
                """event handler for the keyrelease event on this widget"""
                if event.keysym == "BackSpace":
                        self.delete(self.index(Tkinter.INSERT), Tkinter.END)
                        self.position = self.index(Tkinter.END)
                if event.keysym == "Left":
                        if self.position < self.index(Tkinter.END): # delete the selection
                                self.delete(self.position, Tkinter.END)
                        else:
                                self.position = self.position-1 # delete one character
                                self.delete(self.position, Tkinter.END)
                if event.keysym == "Right":
                        self.position = self.index(Tkinter.END) # go to end (no selection)
                if len(event.keysym) == 1:
                        self.autocomplete()
                # No need for up/down, we'll jump to the popup
                # list at the position of the autocompletion

def test(test_list):
        """Run a mini application to test the AutocompleteEntry Widget."""
        root = Tkinter.Tk(className=' AutocompleteEntry demo')
        entry = AutocompleteEntry(root)
        entry.set_completion_list(test_list)
        entry.pack()
        entry.focus_set()
        combo = AutocompleteCombobox(root)
        combo.set_completion_list(test_list)
        combo.pack()
        combo.focus_set()
        # I used a tiling WM with no controls, added a shortcut to quit
        root.bind('<Control-Q>', lambda event=None: root.destroy())
        root.bind('<Control-q>', lambda event=None: root.destroy())
        root.mainloop()

if __name__ == '__main__':
        test_list = ('apple', 'banana', 'CranBerry', 'dogwood', 'alpha', 'Acorn', 'Anise' )
        test(songlist)

NameError: name 'Tkinter' is not defined

In [1]:
top100

NameError: name 'top100' is not defined

In [2]:
Top100

NameError: name 'Top100' is not defined

In [8]:
songlist

["'Til You Can't - Cody Johnson",
 "22 (Taylor's Version) - Taylor Swift",
 'After Last Night - Silk Sonic (Bruno Mars & Anderson .Paak) With Thundercat & Bootsy Collins',
 'All I Want For Christmas Is You - Mariah Carey',
 "All Too Well (Taylor's Version) - Taylor Swift",
 'Already Dead - Juice WRLD',
 "Babe (Taylor's Version) (From The Vault) - Taylor Swift",
 'Bad Habits - Ed Sheeran',
 'Bad Man (Smooth Criminal) - Polo G',
 "Beggin' - Maneskin",
 "Begin Again (Taylor's Version) - Taylor Swift",
 'Better Days - NEIKED X Mae Muller X Polo G',
 "Better Man (Taylor's Version) (From The Vault) - Taylor Swift",
 'Big Energy - Latto',
 'Blast Off - Silk Sonic (Bruno Mars & Anderson .Paak)',
 'Bubbly - Young Thug With Drake & Travis Scott',
 'Buy Dirt - Jordan Davis Featuring Luke Bryan',
 'Chasing After You - Ryan Hurd With Maren Morris',
 'Chosen - Blxst & Tyga Featuring Ty Dolla $ign',
 'Cold As You - Luke Combs',
 'Cold Heart (PNAU Remix) - Elton John & Dua Lipa',
 "Come Back...Be Here